# How to configure runtime chain internals

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)
- [Binding runtime arguments](/docs/how_to/binding/)

:::

Sometimes you may want to experiment with, or even expose to the end user, multiple different ways of doing things within your chains.
This can include tweaking parameters such as temperature or even swapping out one model for another.
In order to make this experience as easy as possible, we have defined two methods.

- A `configurable_fields` method. This lets you configure particular fields of a runnable.
  - This is related to the [`.bind`](/docs/how_to/binding) method on runnables, but allows you to specify parameters for a given step in a chain at runtime rather than specifying them beforehand.
- A `configurable_alternatives` method. With this method, you can list out alternatives for any particular runnable that can be set during runtime, and swap them for those specified alternatives.

## Configurable Fields

Let's walk through an example that configures chat model fields like temperature at runtime:

In [1]:
%pip install --upgrade --quiet langchain langchain-openai

import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

Note: you may need to restart the kernel to use updated packages.


StdinNotImplementedError: getpass was called, but this frontend does not support input requests.

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

model.invoke("pick a random number")

AIMessage(content='17', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d5e25ba2-4acf-481e-ab4e-fb1cb6115ca7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12})

Above, we defined `temperature` as a [`ConfigurableField`](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.utils.ConfigurableField.html#langchain_core.runnables.utils.ConfigurableField) that we can set at runtime. To do so, we use the [`with_config`](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.with_config) method like this:

In [3]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='23', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aecbcfab-06e6-4dfd-b097-014c0b06649a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12})

Note that the passed `llm_temperature` entry in the dict has the same key as the `id` of the `ConfigurableField`.

We can also do this to affect just one step that's part of a chain:

In [4]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

chain.invoke({"x": 0})

AIMessage(content='27', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d7fe6c19-a0cb-407f-b161-ed98d15f1807-0', usage_metadata={'input_tokens': 14, 'output_tokens': 1, 'total_tokens': 15})

In [5]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='79', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4eeffc7f-d741-471e-be71-f805211e9b01-0', usage_metadata={'input_tokens': 14, 'output_tokens': 1, 'total_tokens': 15})

### With HubRunnables

This is useful to allow for switching of prompts

In [6]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

prompt.invoke({"question": "foo", "context": "bar"})

/Users/bagatur/langchain/.venv/lib/python3.11/site-packages/langsmith/client.py:5519: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:", additional_kwargs={}, response_metadata={})])

In [7]:
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]", additional_kwargs={}, response_metadata={})])

## Configurable Alternatives



The `configurable_alternatives()` method allows us to swap out steps in a chain with an alternative. Below, we swap out one chat model for another:

In [8]:
%pip install --upgrade --quiet langchain-anthropic

import os
from getpass import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass()

Note: you may need to restart the kernel to use updated packages.


StdinNotImplementedError: getpass was called, but this frontend does not support input requests.

In [9]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(
    model="claude-3-haiku-20240307", temperature=0
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

# By default it will call Anthropic
chain.invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to come up with a silly pun involving bears. Hopefully you found it at least mildly amusing. Let me know if you'd like to hear another bear-themed joke.", additional_kwargs={}, response_metadata={'id': 'msg_018ZbeSyjj9Er71NyZtBwLhp', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 72}}, id='run-f7c8c0f0-1e8a-4b44-a7ce-2e36f696333a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 72, 'total_tokens': 85})

In [10]:
# We can use `.with_config(configurable={"llm": "openai"})` to specify an llm to use
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-607f40c0-fecd-426b-abed-4860d432c8c5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34})

In [11]:
# If we use the `default_key` then it uses the default
chain.with_config(configurable={"llm": "anthropic"}).invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to come up with a silly pun involving bears. Hopefully you found it at least mildly amusing. Let me know if you'd like to hear another bear-themed joke.", additional_kwargs={}, response_metadata={'id': 'msg_013S2ySwMd4uinEASJenB7Sh', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 72}}, id='run-72272a2a-2d1e-4ba1-bb2e-0d35b9d80ca9-0', usage_metadata={'input_tokens': 13, 'output_tokens': 72, 'total_tokens': 85})

### With Prompts

We can do a similar thing, but alternate between prompts


In [12]:
llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default prompt (asking for a joke, as initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

# By default it will write a joke
chain.invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to come up with a silly pun involving bears. Hopefully you found it at least mildly amusing. Let me know if you'd like to hear another bear-themed joke.", additional_kwargs={}, response_metadata={'id': 'msg_01SHjVVHx9aGZA5ryphMMDiK', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 72}}, id='run-e387e041-804f-4e22-823d-97d1faafd4b0-0', usage_metadata={'input_tokens': 13, 'output_tokens': 72, 'total_tokens': 85})

In [13]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="Here is a short poem about bears:\n\nMajestic bears, strong and true,\nRoaming the forests, wild and free.\nPowerful paws, fur soft and brown,\nCommanding respect, a sight to see.\n\nForaging for berries, fishing streams,\nProtecting their young, a fierce display.\nSymbols of nature's untamed grace,\nBears captivate us, day by day.\n\nMighty and gentle, bears inspire,\nReminding us of Earth's precious might.\nIn their presence, we stand in awe,\nHumbled by their primal, noble plight.", additional_kwargs={}, response_metadata={'id': 'msg_01HgwAsH1LH1eYfoEey7s1Bv', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 140}}, id='run-99848f28-e303-46d1-a5b0-77cae05f214b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 140, 'total_tokens': 153})

### With Prompts and LLMs

We can also have multiple things configurable!
Here's an example doing that with both prompts and LLMs.

In [14]:
llm = ChatAnthropic(
    model="claude-3-haiku-20240307", temperature=0
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default prompt (asking for a joke, as initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

# We can configure it write a poem with OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

AIMessage(content='In the forest deep and wild,\nWhere the trees stand tall and proud,\nRoams a creature fierce and mild,\nThe bear, so strong and loud.\n\nWith fur as dark as night,\nAnd eyes that gleam like gold,\nThey roam the woods with might,\nA sight to behold.\n\nWith a lumbering gait they tread,\nThrough the undergrowth they roam,\nTheir presence fills the forest with dread,\nYet they are truly at home.\n\nSo let us admire these majestic beasts,\nWith their power and their grace,\nFor in the wild, they are the least,\nYet in our hearts, they hold a place.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 13, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-decb0bbb-3cb2-47ed-8bf6-a18bbcaba4cd-0', usage_metadata={'input_tokens': 13, 'output_tokens': 124, 'total_tokens': 137})

In [15]:
# We can always just configure only one if we want
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears like fast food? Because they can't catch it!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-661e5727-09f2-41cd-a1b0-be98a5ba943e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 15, 'total_tokens': 28})

### Saving configurations

We can also easily save configured chains as their own objects

In [16]:
openai_joke = chain.with_config(configurable={"llm": "openai"})

openai_joke.invoke({"topic": "bears"})

AIMessage(content='Why did the bear dissolve in water?\n\nBecause it was polar!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f1b068d2-ac7c-484a-a9c1-45b468cff548-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26})

## Next steps

You now know how to configure a chain's internal steps at runtime.

To learn more, see the other how-to guides on runnables in this section, including:

- Using [.bind()](/docs/how_to/binding) as a simpler way to set a runnable's runtime parameters